In [5]:
import pandas as pd

# Load the data from the uploaded file
data = pd.read_excel("data/Data_BA.xlsx")

# Display the first few rows of the dataset to get an overview of the columns
data.head()

,Område,Team,Indsatslængde,Indsats,Indsatsparagraf,Leverandør,PrisIAlt,Underretter,Haendelsesdato,BaggrundBarnet,BaggrundHjemmet,alder,alder_ved_hændelse,sag_start,sag_slut,status,personID_serial,caseworker_serial,sag_varighed
0,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,90,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,28091.0,Daginstitution/Dagpleje,2020-07-14,Anden bekymrende adfærd hos barn/ung fx udadre...,Højt konfliktniveau eller vold i hjemmet melle...,6,3,2020-07-14,2021-12-02,lukket,629,1,507.0
1,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,161,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,47375.0,Anden kommunal afdeling,2020-09-18,NaN,NaN,2,0,2020-09-18,2022-12-07,lukket,604,1,811.0
2,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,0,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Børne- og Familiehuset Arild,39.0,Sygehus,2021-05-21,Anden form for omsorgssvigt over for barn/ung¤...,Anden bekymrende adfærd hos forældre¤Misbrug h...,1,0,2021-05-21,2023-09-15,lukket,34,1,848.0
3,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Styrket modt...,239,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU Valby-Vesterbro-Kgs.Eng...,141911.0,Anden offentlig myndighed,2020-03-05,Anden form for omsorgssvigt over for barn/ung,Misbrug hos forældre¤Utilstrækkelig omsorg fra...,2,0,2020-03-05,2021-10-18,lukket,167,2,593.0
4,BBU AF Økonomi og Digitalisering,NaN,123,Psykologisk behandling,"SEL § 52, stk. 3, nr. 3 (behandling af barn/un...",Psykolog Tanja Skov Rønne,49000.0,Politi,2021-01-23,NaN,Betydelig eller varigt nedsat fysisk eller psy...,5,3,2021-01-23,2023-02-17,lukket,671,3,756.0


In [6]:
# Drop rows with missing values in 'PrisIAlt' and 'sag_varighed'
cleaned_data = data.dropna(subset=['PrisIAlt', 'sag_varighed'])

# One-hot encode the categorical control variables
encoded_data = pd.get_dummies(cleaned_data, columns=['Indsats', 'Underretter', 'BaggrundBarnet', 'BaggrundHjemmet'], drop_first=True)

# Check the shape of the cleaned and encoded data
encoded_data.shape

(651, 95)

In [7]:
import statsmodels.api as sm

# Define function to run OLS regression
def run_ols(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    return results.summary()

# Model 1: Y = sag_varighed, X = PrisIAlt
y1 = encoded_data['sag_varighed']
X1 = encoded_data['PrisIAlt']
result1 = run_ols(y1, X1)
print("Model 1:\n", result1)

# Model 2: Y = sag_varighed, X = alder_ved_hændelse
y2 = encoded_data['sag_varighed']
X2 = encoded_data['alder_ved_hændelse']
result2 = run_ols(y2, X2)
print("\nModel 2:\n", result2)

# Model 3: Y = sag_varighed, X = PrisIAlt, alder_ved_hændelse control
y3 = encoded_data['sag_varighed']
X3 = encoded_data[['PrisIAlt', 'alder_ved_hændelse']]
result3 = run_ols(y3, X3)
print("\nModel 3:\n", result3)

# Model 4: Y = sag_varighed, X = PrisIAlt, controls: 'alder_ved_hændelse', 'Indsatslængde', 'Indsats', 'Underretter', 'BaggrundBarnet', 'BaggrundHjemmet'
y4 = encoded_data['sag_varighed']
X4_columns = ['PrisIAlt', 'alder_ved_hændelse', 'Indsatslængde'] + [col for col in encoded_data.columns if col.startswith(('Indsats_', 'Underretter_', 'BaggrundBarnet_', 'BaggrundHjemmet_'))]
X4 = encoded_data[X4_columns]
result4 = run_ols(y4, X4)
print("\nModel 4:\n", result4)


Model 1:
                             OLS Regression Results                            
Dep. Variable:           sag_varighed   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2877
Date:                Mon, 23 Oct 2023   Prob (F-statistic):              0.592
Time:                        14:43:29   Log-Likelihood:                -4906.4
No. Observations:                 651   AIC:                             9817.
Df Residuals:                     649   BIC:                             9826.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        546.6001     21.983     24.86